In [ ]:
#### Cluster Expresso Package ### 
library('NMF')


In [ ]:
#### Clustering types 'NMF' or MontiCarlo Consesnus Clustering 


ClusterExpresso <- function(x)
    
    if(type='NMF') {
  AutoNMF <- function(x){
   x_pos <- nneg(as.matrix(x), method=c("min")) 
    NMF_Estimate <- nmfEstimateRank(x_pos,2:10, nrun=100,) #can add in .options="P"- this enables parallel processing
    #rX <- randomize(x_Pos) # For randomised NMF 
    #rand_ Estiamate  <- nmfEstimateRank(rX, 2:10, nrun=100) 
    rw <- which.max(NMF_Estimate$measures$cophenetic) # This will automatically determine optimal rank based on max copenhentic coefictient
Rank <- NMF_Estimate$measures$rank[rw]
res <- nmf(x_pos, Rank, nrun = 1000) 
    return(res)
}
} else {
  

AutoM3C <- function(x) { 
    x_pos <- nneg(as.matrix(exps_data), method=c("min"))
res <- M3C(x_pos, cores=4, seed = 1234, des = NULL) ##Double check cores, 4 is standard##
   rw <- which.max(res$scores$PAC_REAL)
   Rank <- res$scores$K[rw]
    data <- res$realdataresults[[Rank]]$ordered_data # this is the data
annon <- res$realdataresults[[Rank]]$ordered_annotation # this is the annotation
ccmatrix <- res$realdataresults[[Rank]]$consensus_matrix # this is the consensus matrix
    data_list <- list('data'= data, 'annon' = annon, 'ccmatrix'=  ccmatrix)
    return(data_list)
} 
    }

 ## Cluster_Plots ## 

if(plots='TRUE') {
# normalise and scale the data
data <- t(scale(t(data_list$data))) # z-score normalise each row (feature)
data <- apply(data, 2, function(x) ifelse(x > 4, 4, x)) # compress data within range
data <- apply(data, 2, function(x) ifelse(x < -4, -4, x)) # compress data within range

# get some cool colour palettes from the ggsci package and RColourBrewer
ann_colors <- ggsci::pal_startrek("uniform")(4) # star trek palette
ann_colors2 <- ggsci::pal_futurama()(4) # futurama palette
pal <- rev(colorRampPalette(RColorBrewer::brewer.pal(10, "RdBu"))(256))
              NMF::aheatmap(data, annCol = data_list$annon, Colv = NA, distfun = 'pearson', 
         color = gplots::bluered(256), annColors = list(class=ann_colors, consensuscluster=ann_colors2))


n <- 10
seq = rev(seq(0,255,by=255/(n)))
palRGB = cbind(seq,seq,255)
mypal <-rgb(palRGB,maxColorValue=255)
# plot consensus matrix heatmap, do not cluster rows and columns
NMF::aheatmap(data_list$ccmatrix, 
              annCol = data_list$annon[,1,drop=FALSE], 
              color = mypal, scale = 'none', cexRow = 0, cexCol = 0,
              Colv=NA,Rowv=NA,annColors = list(consensuscluster=ann_colors2))
              
              }
              
## DE Analysis ##
              DEA <- function(data_list$data, data_list$annon)
                  {if(DEA=TRUE) 
                  ArrayData <- loadMAdata(dataNorm=data_list$data, setup=data_list$annon)
              